Fazendo a primeira chamada da LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

import pandas as pd
from langchain.docstore.document import Document

True

Criando nossa primeira chamada de LLM

In [2]:
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash', google_api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
llm.invoke('Quem foi Albert Einstein?').content

'Albert Einstein (1879-1955) foi um físico teórico alemão que desenvolveu a teoria da relatividade, um dos dois pilares da física moderna (o outro é a mecânica quântica).  Sua famosa equação, E=mc², que estabelece a equivalência entre energia e massa, é uma das mais conhecidas do mundo.\n\nAlém da relatividade, Einstein fez contribuições significativas em outras áreas da física, incluindo a mecânica estatística (explicação do movimento browniano), a cosmologia e a mecânica quântica (embora tivesse reservas sobre alguns de seus aspectos).  Ele recebeu o Prêmio Nobel de Física de 1921 por sua explicação do efeito fotoelétrico, que ajudou a estabelecer a teoria quântica da luz.\n\nEinstein é considerado um dos maiores e mais influentes cientistas de todos os tempos, não apenas por suas descobertas científicas, mas também por seu impacto na filosofia e na cultura popular. Sua imagem icônica, com o cabelo despenteado e a língua para fora, é mundialmente reconhecida. Ele também era conhecido

Construindo nossa primeira estrutura de RAG

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
BASE_DIR = Path().resolve()
DOCS_DIR = BASE_DIR / "docs"

Escolhendo a estrutura de embedding

In [6]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\degio\AppData\Local\Temp\ipykernel_21656\323160131.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\degio\Downloads\2025\Preditiva\Formação AI\Conteudos\Módulo 8 - Projetos Finais\CASE 1\Assistente_AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
def load_pdf_vectorstore(filepath : str, save_path: str):
    loader = PyPDFLoader(DOCS_DIR / filepath)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    documents = text_splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(documents, embedding)
    vectorstore.save_local(f'vectorstores/{save_path}')
    retriver = vectorstore.as_retriever(search_type = 'similarity', search_kwargs = {'k':7})
    return retriver
    

In [36]:
retriver_perguntas_frequentes = load_pdf_vectorstore("Perguntas Frequentes.pdf", "vectorstore_perguntas_frequentes")
retriver_manual_tecnico = load_pdf_vectorstore("Manual Tecnico de Produtos.pdf", "vectorstore_manual_tecnico_produtos")
retriver_politicas_procedimentos = load_pdf_vectorstore("Politicas e Procedimentos.pdf", "vectorstore_politicas_procedimentos")

In [49]:
def load_excel_vectorstore(filepath : str, save_path: str):
    df = pd.read_excel(DOCS_DIR / filepath)
    documents = []
    for idx, row in df.iterrows():
        text = ' '.join([str(cell) for cell in row if pd.notna(cell)])
        documents.append(Document(page_content = text, metadata ={'row': idx}))

    vectorstore_tickers = FAISS.from_documents(documents, embedding)
    vectorstore_tickers.save_local('vectorstores/vectorstore_tickets')
    retriver_tickets = vectorstore_tickers.as_retriever(search_type = 'similarity', search_kwargs = {'k':7})

    return retriver_tickets

In [50]:
retriver_tickets = load_excel_vectorstore("Tickets.xlsx", "vectorstore_tickets")